# Globals and Imports

In [ ]:
import os
import sys
import re
import whois
# print sys.executable
# print sys.path
# from jupyter_core.paths import jupyter_data_dir
# print(jupyter_data_dir())
import dnstwist
import string
from tld import get_tld
from email import parser, message_from_string
from dns import resolver

# Helper Functions

In [67]:
# Parse email address
def parse_email(text):
    text = text.decode('utf-8')
    match = re.findall(r'[\w\.-]+@[\w\.-]+', text, re.UNICODE)
    return match

# Determine if a host domain is good 
def is_domain_legit(url):
    good_domain = False
    while url.find('.') > 0:
        try:
            answers = resolver.query(url, rdtype="SOA")
            for rdata in answers:
                # print (rdata)
                good_domain = True
            break
        except:
            url = url[url.find('.') + 1:]
    return good_domain

# Determine if a domain is spoofed
def is_domain_spoofed(brands, target):
    domain_spoofed = False
    cwd = os.getcwd() + '/data/'
    for brand in brands:
        with open (cwd + brand + '.fuzz', 'r') as rf:
            lines = rf.readlines()
            domain = lines[0]
            for line in lines[1:]:
                if target.encode('utf-8') in line:
                    domain_spoofed = True
                    break
        if domain_spoofed:
            break
    return domain_spoofed, domain

# TLD Top Level Domain
def get_top_level_domain(url):
    res = get_tld("http://www.yahoo.co.jp", as_object=True)
    #     print (res)
    #     print (res.domain)
    #     print (res.subdomain)
    print (res.tld)
    #     print (res.suffix)
    return res.tld


# Instantiate an Email Object with Email Header Texts

In [68]:
# from email import parser, message_from_string
# texts = """DKIM-Signature: v=1; a=rsa-sha256; c=relaxed/relaxed; d=yahoo-inc.com; s=yibm; t=1465479679; 
# To: test@yahoo.com 
# From: "Yahoo" 
# Reply-To: "Yahoo" 
# X-YMailISG: PCypxycWLDvGv4Bg8ShrtzVYi3vpFMAjYaqWyWybcVJ_ZQff eyquyqb..Qu6UKhX_Tyz5b3da2iDtRStJpVnNulZHOb8GznJQTCKk9sjvboS KsbzY4E1uScWz0Ieo0jjG0YHrB1dTCzOSeMiPNumCCFS1sR3_SkyMBGG_D2D wWtdRducxLa2YgEMMubVpMtNJMBv.bwk0.E.jQNEy8I3LnJEqcDpmIUM7bZL XgkEFz7yl1Zo6Sj4r0z6pGlVIFOql7uG9Bwq2VJoK1Q1upKJUOBfQqzf64y2 9fXLnQsWENpZloxwncGzLhdzEYGgE3xNuFV8QFxZGXyvtKZFoykH49M03URN jtx8Yg6ypjyRbBIRVJGVFbjAvW6io3yeyIFh042jlgYQtLxbneFA60hn9ifT Mit3bQ5l7Tginw0OgRM2cbqLo0tEZFt9vlN597Z3vPGwsVdBcTp9wnk6orj2 TqjEpAmODy3Yru2HzDP7Dbwq9CGaIozUm91VNWqw5Dy7AMQEsuvnBop7Fflk G21m1WKMBgrS.2bOLQ4797E09LjlyyoWI9FouUNNhDljnPPf2AeKUKzauctw ULOQPveWAm4lDsNLMp5yvXDYNIe5HMor84SVd8_xF3Icna1PAftXGzJUHrXK NZSEN_VO0GprGfaNQg4uSW_0wXFXwC6TYQ4CMjz53o0qNGpILogVfRLwFCFL DtW8nimkLLsNzmDajzJsR_juA86Orw2NE5ED4qdpPxmyxyrXYOQPu3O6zeYf 7mBzU0aX7VHJUxJ4L3HdB9qTjbTaCdnySrnjGtd7u9Cn9yRJirDNeg3UA82P PeA1ZDfc0vKdrn5QI6e6YKa2TTt7Dspy3jObgSapH5epc3LyQVyN7yjpxrq_ MXAbpqedjUfcwq3c7lpt8xxUxy.MXWg0fJO059xijvb_sYTaQTGUWAMeVU.6 IW.hSksejwpn._CgE9Kqabbk5qgYIdYRW1pmz5OBYh0skCX1TrFRuxbGvDit R_wr.wbTpJGiSST.b0ZetmgN72bVvlRtmNPw1Dk.zxaacXxhGSMWupPUDLJZ OMrap2ax8oiQrxT3jIhk8seIkaNJ.tGUhlPx6G4lJJaz0g89LmjBaEjGUG8P W3Phh9db3hjxUIX5UC0jg5ai2XZ7u_wXn2Muk61N1eRCZ0oA2S25YDPK1dh. 3VQ6pH8SSBxVkQHUJXbZUNqLAzi5V5wRS7oeitXERGgA2DiZB268.rJxS7di OMT5eGoITG4LnAo1M3nsVQ6xceHDd4v6KD9KfBgTHX_iLUv_skCv4dVUgVvj edKOFiOMHBTpJ9J9BECjTTzEUpc.fCNUcRwSsiSkqbRhUsAdCbxQZir3Nb1Z 6FzI6J2eNqpj4azjmDeI15R8MyN7VFc6bl6pCZySk2Tx5SQESDm.sVkADSVR pI2nuscEjU3xo_qGUxbh5mbAA17K2zYpcFXaOce8_9Eszos5pURCcdtBYUqI I_DOtvNe.zWY1ShRcr9ZzTj3ibmc7NBmvumhVMjqirb12mfJ6oxHv8d86gze HtAJmJghczUg5otSzdxSgEJJxjMZrzSidJ9FP.gPiPWtuukz82YpZ32MnCVs 6.V2DRxpUmZa31KH93QSEzwMlCn3FFTLBv9izcjoFP81yeAn.3QloF8XIC3K WmtXtloyeGjuygAhlkd_prXmMGGC5JmPlY8xu4k1NavkdDh6pG6zIkt83Wsd p.D.0BgM 
# X-Originating-IP: [75.30.245.45] 
# Authentication-Results: mta1287.mail.ne1.yahoo.com from=yahoo-inc.com; domainkeys=neutral (no sig); from=yahoo-inc.com; dkim=pass (ok)
# X-Apparently-To: test@yahoo.com; Thu, 09 Jun 2016 13:41:21 +0000 
# Return-Path: 
# Received-SPF: pass (domain of yahoo.com designates 72.30.235.45 as permitted sender) 
# Received: from 127.0.0.1 (EHLO n3-vm9.bullet.mail.bf1.yahoo.com) (72.30.235.45) by mta1287.mail.ne1.yahoo.com with SMTPS; Thu, 09 Jun 
# 2016 13:41:21 +0000 
# """
texts="""content-class: urn:content-classes:message 
Subject: Warrants on $25 Million Funding 
Date: Thu, 3 Aug 2000 14:43:47 -0500 
MIME-Version: 1.0
Content-Type: application/ms-tnef; name="winmail.dat"
Content-Transfer-Encoding: binary
Message-ID: <3989e793.87BDEEE2@tech.com>
X-MS-Has-Attach:
X-MS-TNEF-Correlator: <3989e793.87BDEEE2@tech.com> 
Thread-Topic: Warrants on $25 Million Funding 
Thread-Index: AcHatCZUSkaLe0ajEdaelQACpYcy8A==
From: "CEO Good_Guy@ibм.com" <ceo_good_guy@yahoo.com.jp >
To: "Bad_Guy_Broker" <bad_guy@fund.com
Return-Path: CEO Good_Guy@tech.com
Received: from mail.tech.com (mail.tech.com [201.10.20.152]) by hedgefund.fund.com (8.11.0/8.11.0) e73MfZ331592;
Received: from webmail.tech.com (webmail.tech.com [10.27.30.190]) by mail.tech.com (Switch-2.0.1/Switch- 2.0.1)     ; Thu, 3 Aug 2000
Received: from tech.com (ostrich.tech.com [10.27.20.190]) by webmail.tech.com (8.8.8+Sun/8.8.8) with ESMTP id RAA01318; Thu, 3 Aug 2000 14:41:31 -0500
"""
# msg = message_from_string(headers)
# print(msg.keys())
headers = parser.Parser().parsestr(texts, headersonly=True)
# for key in headers.keys():
print (headers)

From nobody Wed May 23 13:16:28 2018
content-class: urn:content-classes:message 
Subject: Warrants on $25 Million Funding 
Date: Thu, 3 Aug 2000 14:43:47 -0500 
MIME-Version: 1.0
Content-Type: application/ms-tnef; name="winmail.dat"
Content-Transfer-Encoding: binary
Message-ID: <3989e793.87BDEEE2@tech.com>
X-MS-Has-Attach: 
X-MS-TNEF-Correlator: <3989e793.87BDEEE2@tech.com> 
Thread-Topic: Warrants on $25 Million Funding 
Thread-Index: AcHatCZUSkaLe0ajEdaelQACpYcy8A==
From: "CEO Good_Guy@ibм.com" <ceo_good_guy@yahoo.com.jp >
To: "Bad_Guy_Broker" <bad_guy@fund.com
Return-Path: CEO Good_Guy@tech.com
Received: from mail.tech.com (mail.tech.com [201.10.20.152]) by
 hedgefund.fund.com (8.11.0/8.11.0) e73MfZ331592; 
Received: from webmail.tech.com (webmail.tech.com [10.27.30.190]) by
 mail.tech.com (Switch-2.0.1/Switch- 2.0.1) ; Thu, 3 Aug 2000
Received: from tech.com (ostrich.tech.com [10.27.20.190]) by webmail.tech.com
 (8.8.8+Sun/8.8.8) with ESMTP id RAA01318; Thu, 3 Aug 2000 14:41:31 -050

# Read in selected lists of brand names and phishing words

In [69]:
target_brands = []
phish_words = []

print (os.getcwd())
with open(os.getcwd() + "/data/brands.txt") as fh:
    for line in fh:
        name = line.strip().translate(None, string.whitespace)
        name_fuzz = os.getcwd() + "/data/" + name + ".fuzz"
        target_brands.append(name)
        if not os.path.isfile(name_fuzz):
            fuzz = dnstwist.DomainFuzz(name+".com")
            fuzz.generate()
            with open(name_fuzz, 'w') as wf:
                for domain in fuzz.domains:
                    wf.write(domain['domain-name'].encode('utf-8')+'\n')
        
with open(os.getcwd() + "/data/phish_words.txt") as fh:
    for line in fh:
        phish_words.append(line.strip().encode('utf-8'))

print target_brands
print phish_words

/Users/emoryshyong/Documents/PyCharm/project/dnstwist
['apple', 'google', 'microsoft', 'facebook', 'coco-cola', 'amazon', 'disney', 'toyota', 'mcdonalds', 'samsung', 'ge', 'att', 'ibm', 'intel', 'nike', 'cisco', 'mercedes-benz', 'oracle', 'verizon', 'louisvuitton', 'bmw', 'budweiser', 'americanexpress', 'walmart', 'marlboro', 'honda', 'sap', 'visa', 'gillette', 'pepsi', 'nescafe', 'espn', 'loreal', 'homedepot', 'hm', 'starbuck', 'audi', 'accenture', 'ford', 'fritolay', 'ikea', 'hp', 'wellsfargo', 'hermes', 'ups', 'cvs', 'guuci', 'pamper', 'siemens', 'hsbc', 'zara', 'mastercard', 'jpmorgan', 'nestle', 'eBay', 'fox', 'chevrolet', 'danone', 'colgate', 'porsche', 'cartier', 'citi', 'kraft', 'lexus', 'chase', 'banksofamerica', 'nissan', 'hyundai', 'rolex', 'redbull', 'netflix', 'kellog', 'sony', 'santander', 'adidas', 'lego', 'basf', 'heineken', 't-mobile', 'bbva', 'corona', 'caterpillar', 'fedex', 'adobe', 'goldmansachs', 'lowes', 'channel', 'huawei', 'target', 'phillips', 'boeing', 'subwa

# Check suspicousness by the subject line 

In [70]:
subject_line = headers['Subject']
ss = subject_line.split()
print (ss)
suspicious_subject = False
for s in ss:
    s = s.lower()
    if s in target_brands:
        suspicious_subject = True
        break
    else:
        for w in phish_words:
            if w in s:
                suspicious_subject = True
                break
if suspicious_subject:
    print ('Subject line is suspicous')


['Warrants', 'on', '$25', 'Million', 'Funding']
Subject line is suspicous


# Check suspicousness by sender's email

In [71]:
from_email = headers['From']
print (from_email)
email_parsed = parse_email(from_email)
print (email_parsed)

suspicious_sender = False
if len(set(email_parsed)) > 1:
    suspicious_sender = True
if suspicious_sender:
    print ('Sender has two different email identities')
    
delchars = ''.join(c for c in map(chr, range(256)) if not c.isalnum())
# python 3
# scrunched = from_email.translate({ord(c):'' for c in delchars})
# python 2
scrunched = from_email.translate(None, delchars)
scrunched = scrunched.lower()
# print (scrunched)
suspicious_name = False
for b in target_brands:
    if b in scrunched:
        suspicious_name = True
        break
if not suspicious_name:
    for w in phish_words:
        if w in scrunched:
            suspicious_name = True
            break
if suspicious_name:
    print('Name of business was one of the most targeted companies')

"CEO Good_Guy@ibм.com" <ceo_good_guy@yahoo.com.jp >
[u'Good_Guy@ib\u043c.com', u'ceo_good_guy@yahoo.com.jp']
Sender has two different email identities


# Identify Legitimate Email Host Domains 

In [76]:
# myregex = r'(?:[a-zA-Z0-9](?:[a-zA-Z0-9\-]{,61}[a-zA-Z0-9])?\.)+[a-zA-Z]{2,6}'
myregex = r'(?:[(\w+)](?:[(\w+)\-]{,61}[(\w+)])?\.)+[a-zA-Z]{2,6}'
sender_domains = []
suspicous_email_domain = False
for email in email_parsed:
    print email
    domains = re.findall(myregex, email, re.UNICODE)
    for domain in domains:
        # print domain
        flag, target = is_domain_spoofed(target_brands, domain)
        if flag:
            print ('{0} is a spoof of {1}'.format(domain.encode('utf-8'), target.strip()))
        flag = is_domain_legit(domain)
        if flag:
            print ('{} is a legitimate Top Level Domain'.format(domain.encode('utf-8')))
        else:
            print ('{} is not a legitimate Top Level Domain'.format(domain.encode('utf-8')))
        if flag:
            details = whois.query(domain)
#             print (details.__dict__)
#             print ('Name is {}'.format(details.name))
            print ('Registrar is {}'.format(details.registrar))
#             print ('Creation_date is {}'.format(details.creation_date))
#             print ('Expiration_date is {}'.format(details.expiration_date))
#             print ('Updated_date is {}'.format(details.last_updated))
        else:
            suspicous_email_domain = True


Good_Guy@ibм.com
ibм.com is a spoof of ibm.com
ibм.com is not a legitimate Top Level Domain
ceo_good_guy@yahoo.com.jp
yahoo.com.jp is not a legitimate Top Level Domain


# Identify Legitimate Received Host Domains

In [77]:
received_email_servers = headers['Received']
myregex = r'(?:[(\w+)](?:[(\w+)\-]{,61}[(\w+)])?\.)+[a-zA-Z]{2,6}'
# print (received_email_servers)
suspicous_received_domain = False
received_domains = re.findall(myregex, received_email_servers, re.UNICODE)
received_domains = set(received_domains)
# print (received_domains)
for domain in received_domains:
    flag = is_domain_legit(domain)
    if flag:
        details = whois.query(domain)
        print ('{0} of {1} is a legitimate Top Level Domain'.format(details.name, domain)) 
#         print (details.__dict__)
#         print ('Name is {}'.format(details.name))
        print ('Registrar is {}'.format(details.registrar))
#         print ('Creation_date is {}'.format(details.creation_date))
#         print ('Expiration_date is {}'.format(details.expiration_date))
#         print ('Updated_date is {}'.format(details.last_updated))
    else:
        suspicous_received_domain = True
        print ('{} does not contain a legitimate Top Level Domain'.format(domain))

tech.com of mail.tech.com is a legitimate Top Level Domain
Registrar is Network Solutions, LLC.
tech.com of (mail.tech.com is a legitimate Top Level Domain
Registrar is Network Solutions, LLC.
fund.com of hedgefund.fund.com is a legitimate Top Level Domain
Registrar is DNC Holdings, Inc.


# Identify Top Level Domain

In [ ]:
from tld import get_tld
from dns import resolver
res = get_tld("http://www.yahoo.co.jp", as_object=True)
print (res)
print (res.domain)
print (res.subdomain)
print (res.tld)
print (res.suffix)

# Collect Hostname Details by WHOIS

In [34]:
# import whois
domains = ['google.com', 'stackoverflow.com', 'clearedin.com', 'expediamail.com']
for domain in domains:
    details = whois.query(domain)
    print (details.name, details.registrar, details.creation_date, details.expiration_date)
    print (details.name_servers)

google.com MarkMonitor Inc. 1997-09-15 04:00:00 2020-09-13 21:00:00-07:00
{'ns1.google.com\r', 'ns3.google.com', 'ns2.google.com\r', 'ns4.google.com', 'ns2.google.com', 'ns4.google.com\r', 'ns3.google.com\r', 'ns1.google.com'}
stackoverflow.com Name.com, Inc. 2003-12-26 19:18:07 2019-02-02 11:59:59
{'ns-cloud-e2.googledomains.com\r', 'ns-cloud-e1.googledomains.com', 'ns-358.awsdns-44.com\r', 'ns-cloud-e1.googledomains.com\r', 'ns-1033.awsdns-01.org', 'ns-358.awsdns-44.com', 'ns-cloud-e2.googledomains.com', 'ns-1033.awsdns-01.org\r'}
clearedin.com Google Inc. 2017-08-08 15:12:38 2018-08-08 15:12:38
{'ns-1789.awsdns-31.co.uk\r', 'ns-733.awsdns-27.net', 'ns-1159.awsdns-16.org\r', 'ns-733.awsdns-27.net\r', 'ns-1789.awsdns-31.co.uk', 'ns-1159.awsdns-16.org', 'ns-452.awsdns-56.com\r', 'ns-452.awsdns-56.com'}
expediamail.com CSC Corporate Domains, Inc. 2003-12-06 19:16:08 2019-12-06 19:16:08
{'pdns2.ultradns.net', 'pdns5.ultradns.info', 'pdns4.ultradns.org\r', 'pdns6.ultradns.co.uk', 'pdns6.u

# Collect Hostname Details by PYTHONWHOIS

In [ ]:
import pythonwhois
domains = ['google.com', 'stackoverflow.com', 'clearedin.com']
for dom in domains:
    details = pythonwhois.get_whois(dom)
    print (details['status'])